# E2E NLG Challenge

This is a sequece2sequence model transfering meaning representations(MR) of a restaurant into a description in natural language(NL). Pytorch is used to build up the seq2seq neural model.

This is based on [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) by Sean Robertson from the PyTorch website. It has been modified by Katy Ilonka Gero for COMS W4705 taught at Columbia University by Professor Kathy McKeown.

### Overview

The aim of this jupyter notebook is to:

1. train a model on the E2E restaurant data set on a GPU
2. implement beam search for evaluation
3. implement a BLEU evaluator and report BLEU scores

In this dataset, the inputs are restaurant meaning representations, which are a series of key-value pairs that encode information about a restaurant. The outputs are fluent sentences that describe the restaurant. Here is an example:

*Input: Meaning Representation*

```
name[The Eagle],
eatType[coffee shop],
food[French],
priceRange[moderate],
customerRating[3/5],
area[riverside],
kidsFriendly[yes],
near[Burger King]
```

*Output: Fluent Sentence*

```
The three star coffee shop, The Eagle, gives families a mid-priced dining experience featuring a variety of wines and cheeses. Find The Eagle near Burger King.
```

In [1]:
import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext

In [2]:
# this is useful for checking if you are successfully using the GPU

mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

device(type='cuda')

In [3]:
### Define the length filter
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 50

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### 1. Load the e2e data

In [5]:
# LOADING THE E2E DATA HERE

train_path = 'data/e2e-dataset/trainset.csv'
dev_path = 'data/e2e-dataset/devset.csv'

src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

Have a look at the vocab and some example data points.

In [6]:
src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

20 tokens from input vocab:
 ['<unk>', '<pad>', 'customer', 'name[The', 'of', 'area[riverside],', 'out', '5],', 'eatType[coffee', 'shop],', 'than', 'familyFriendly[yes]', 'familyFriendly[yes],', 'area[city', 'eatType[pub],', 'centre],', 'food[Japanese],', 'food[Italian],', 'food[Fast', 'food],']

20 tokens from output vocab:
 ['<unk>', '<pad>', 'is', '<eos>', '<sos>', 'a', 'The', 'the', 'in', 'near', 'of', 'and', 'food', 'customer', 'located', 'It', 'restaurant', 'has', 'coffee', 'price']

num training examples: 42038

example train data:
src:
 ['name[The', 'Waterman],', 'food[Italian],', 'priceRange[high],', 'customer', 'rating[average],', 'area[city', 'centre],', 'familyFriendly[yes]']
tgt:
 ['<sos>', 'The', 'Waterman', 'is', 'a', 'children', 'friendly', 'Italian', 'restaurant', 'in', 'the', 'city', 'centre', 'that', 'has', 'an', 'average', 'rating', 'and', 'high', 'prices.', '<eos>']


### Model definition and training functions

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [8]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    # create a variable for loss
    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [9]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01, teacher_forcing_ratio=0.5):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    

    criterion = nn.NLLLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            

            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')

### 2. Create and train a model

In [10]:
hidden_size = 128
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)

In [34]:
trainIters(encoder1, decoder1, 1, print_every=1000, teacher_forcing_ratio=0.75)

Running 1 epochs...
step: 1000	 avg loss: 4.34	 time for 1000 steps: 0.45 min
step: 2000	 avg loss: 3.53	 time for 1000 steps: 0.43 min
step: 3000	 avg loss: 3.34	 time for 1000 steps: 0.42 min
step: 4000	 avg loss: 3.23	 time for 1000 steps: 0.42 min
step: 5000	 avg loss: 3.14	 time for 1000 steps: 0.43 min
step: 6000	 avg loss: 3.02	 time for 1000 steps: 0.43 min
step: 7000	 avg loss: 2.93	 time for 1000 steps: 0.43 min
step: 8000	 avg loss: 2.84	 time for 1000 steps: 0.44 min
step: 9000	 avg loss: 2.88	 time for 1000 steps: 0.43 min
step: 10000	 avg loss: 2.82	 time for 1000 steps: 0.43 min
step: 11000	 avg loss: 2.81	 time for 1000 steps: 0.43 min
step: 12000	 avg loss: 2.77	 time for 1000 steps: 0.43 min
step: 13000	 avg loss: 2.76	 time for 1000 steps: 0.43 min
step: 14000	 avg loss: 2.64	 time for 1000 steps: 0.42 min
step: 15000	 avg loss: 2.71	 time for 1000 steps: 0.43 min
step: 16000	 avg loss: 2.65	 time for 1000 steps: 0.44 min
step: 17000	 avg loss: 2.58	 time for 1000 st

In [11]:
# Saving and loading the model
torch.save(encoder1.state_dict(), 'encoder.mdl')
torch.save(decoder1.state_dict(), 'decoder.mdl')

hidden_size = 128
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)
encoder1.load_state_dict(torch.load('encoder.mdl'))
decoder1.load_state_dict(torch.load('decoder.mdl'))

<All keys matched successfully>

In [12]:
# Evaluation using Greedy Search Algorithm Decoder
def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

### 3. Implement beam search evaluator

Be sure to return all the output sequences (i.e. if the beam size is k, you should return k sequences) and their associated probabilities. You will need the associated probabilities to select the best performing sequence when calculating BLEU.

In [102]:
# Evaluation using Beam Search Decoder

# The output of next cell should be an example input from the dev set, 
# and three outputs from a beam search evaluator.

def beam_search_evaluate(encoder, decoder, sentence, max_length=MAX_LEN, k = 3):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        seq_list = [([],0,decoder_input,decoder_hidden)]
        alpha = .8    ###parameter for length penalty
        for di in range(max_length):
            cand = []
            for seq, score, decoder_input,decoder_hidden in seq_list:
                if output_vocab.itos[decoder_input] != EOS_TOKEN: 
                    for i in range(k):                   
                        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                        topv, topi = decoder_output.data.topk(k)
                        value = topv[0][i]
                        idx = topi[0][i]
                        lp = (5+len(seq)+1)**alpha/(5+1)**alpha
                        cand.append(tuple([seq+[output_vocab.itos[idx]], (score+value)/lp, idx.squeeze().detach(), decoder_hidden]))
                else:
                    lp = lp = (5+len(seq))**alpha/(5+1)**alpha
                    cand.append(tuple([seq, (score+value)/lp, decoder_input, decoder_hidden]))
            ordered = sorted(cand, key=lambda tup:tup[1], reverse = True)
            seq_list = ordered[:k]

        return [l[0] for l in seq_list]

In [164]:
###Beam Search Output Examples
for i in range(3):
    item = random.choice(data_dev.examples)
    seq = item.src
    print(seq)
    beam_res = beam_search_evaluate(encoder1, decoder1, seq, max_length = 70, k = 3)
    for words in beam_res:
        print(' '.join(words))
    print()

['name[Fitzbillies],', 'eatType[coffee', 'shop],', 'food[Chinese],', 'priceRange[cheap],', 'customer', 'rating[average],', 'area[riverside],', 'familyFriendly[no]']
<sos> Fitzbillies is an inexpensive coffee shop that provides Chinese food in the cheap price range. It is located in the riverside. It is not family friendly and has an average customer rating. It has an average customer rating and has an average customer rating and has an average customer rating and has a low price range. It is located in the riverside area. <eos>
<sos> Fitzbillies is an inexpensive coffee shop that provides Chinese food in the cheap price range. It is located in the riverside. It is not family friendly and has an average customer rating. It has an average customer rating and has an average customer rating and has an average customer rating and has a low price range. are located in the riverside area. <eos>
<sos> Fitzbillies is an inexpensive coffee shop that provides Chinese food in the cheap price range

In [14]:
# This selects 5 random datapoints from the training data and shows the generated sequence

for i in range(5):
    item = random.choice(data_train.examples)
    seq = item.src
    print(seq)
    words = evaluate(encoder1, decoder1, seq)
    print(' '.join(words))
    print()

['name[The', 'Phoenix],', 'food[Indian],', 'priceRange[£20-25],', 'customer', 'rating[high],', 'area[city', 'centre]']
<sos> The Phoenix serves Indian food in the high price range. It is located in the city centre. <eos>

['name[Cotto],', 'food[Indian],', 'customer', 'rating[1', 'out', 'of', '5],', 'familyFriendly[yes],', 'near[Ranch]']
<sos> Cotto is a Indian restaurant that serves Indian food It is friendly rating and is <eos>

['name[Midsummer', 'House],', 'food[Fast', 'food],', 'priceRange[cheap],', 'customer', 'rating[5', 'out', 'of', '5],', 'near[All', 'Bar', 'One]']
<sos> Midsummer House is a cheap restaurant Bar near All Bar One. One. <eos>

['name[The', 'Mill],', 'eatType[pub],', 'food[English],', 'priceRange[high],', 'area[city', 'centre]']
<sos> The Mill is a pub located in the city centre. <eos>

['name[The', 'Golden', 'Palace],', 'eatType[coffee', 'shop],', 'food[Japanese],', 'priceRange[moderate],', 'customer', 'rating[1', 'out', 'of', '5],', 'area[riverside]']
<sos> The 

### 4. Implement BLEU evaluator

Remember that when calculating BLEU using beam search, select the top-scoring sequence output using the model probability.

In [151]:
# THE BLEU EVALUATION HERE

# The output of next 2 cells should be the average BLEU score on the dev set
# for greedy decoding AND for beam search decoding (beam size = 3)
import numpy as np

###Find all the references
def get_ref_list(src):
    ref_list = []
    for e in data_dev.examples:
        if e.src == list(src):
            ref_list.append(e.tgt)
    return(ref_list)

###Model Output (Candidate)
def get_cand(src, decoder_type = "beam"):
    if decoder_type == "beam":
        cand = beam_search_evaluate(encoder1, decoder1, src, max_length = 100, k = 3)[0]
    else:
        cand = evaluate(encoder1, decoder1, src)
    return(cand)

###Calculate the BLEU score
def get_score(src, decoder_type, ngram):
    cand = get_cand(src, decoder_type)
    p_list = []
    ref_list = get_ref_list(src)
    
    #n-gram
    for n in range(1,ngram + 1):
        ref_vocab_list = []

        ###Build up n-gram dict for all the references
        for ref in ref_list:
            ref_vocab = dict()
            for i in range(len(ref)-n+1):
                s = tuple(ref[i:(i+n)])
                if ref_vocab.get(s):
                    ref_vocab[s] += 1
                else:
                    ref_vocab.update({s:1})
            ref_vocab_list.append(ref_vocab)

        ###Build up n-gram dict for the candidate
        cand_vocab = dict()
        for i in range(len(cand)-n+1):
            s = tuple(cand[i:(i+n)])
            if cand_vocab.get(s):
                cand_vocab[s] += 1
            else:
                cand_vocab.update({s:1})

        ###Calculate n-gram precision
        numerator = 0
        for s in cand_vocab:
            max_ref = max([ref_vocab.get(s) if ref_vocab.get(s) else 0 for ref_vocab in ref_vocab_list])
            max_ref = max_ref if max_ref else 0
            numerator += max_ref
        p = numerator/(len(cand)-n+1)
        if n > 1 and p == 0:
            p = p_list[-1]
        p_list.append(p)
    
    ###Brevity Penalty
    c = len(cand)
    len_list = np.array([len(ref) for ref in ref_list])
    r = len_list[np.argmin(abs(len_list-c))]
    BP = max(1,np.exp(1-r/c))
    return BP * np.exp(np.mean(np.log(p_list)))

In [152]:
###BLEU score for beam search
src_unique = list(set([tuple(e.src) for e in data_dev.examples[1:]]))
BLEU_score = []
decoder_type = "beam"
ngram = 4

for i, src in enumerate(src_unique):
    BLEU_score.append(get_score(src, decoder_type, ngram))
    if (i+1) % 100 == 0:
        print("{0} out of {1} completed".format(i+1, len(src_unique)))
print("All completed!")
print(np.mean(BLEU_score))

100 out of 547 completed
200 out of 547 completed
300 out of 547 completed
400 out of 547 completed
500 out of 547 completed
All completed!
0.4760110725346942


In [134]:
###BLEU score for greedy search
greedy_BLEU_score = []
decoder_type = "greedy"
ngram = 4

for i, src in enumerate(src_unique):
    greedy_BLEU_score.append(get_score(src, decoder_type, ngram))
    if (i+1) % 100 == 0:
        print("{0} out of {1} completed".format(i+1, len(src_unique)))
print("All completed!")
print(np.mean(greedy_BLEU_score))

100 out of 547 completed
200 out of 547 completed
300 out of 547 completed
400 out of 547 completed
500 out of 547 completed
All completed!
0.5236403261617786


In [56]:
### For Error Analysis: Unseen Restaurant names
for idx in [10, 25, 300]:
    print(src_unique[idx])
    print(' '.join(get_cand(src_unique[idx], decoder_type = "beam")))
    print()

('name[The', 'Cambridge', 'Blue],', 'eatType[coffee', 'shop],', 'customer', 'rating[1', 'out', 'of', '5],', 'area[riverside],', 'familyFriendly[yes],', 'near[Burger', 'King]')
<sos> The Eagle coffee shop is located near Burger King and has a customer rating of 1 out of 5. It is family friendly and is located in the riverside area. <eos>

('name[The', 'Cricketers],', 'eatType[coffee', 'shop],', 'food[English],', 'customer', 'rating[3', 'out', 'of', '5],', 'familyFriendly[yes],', 'near[The', 'Portland', 'Arms]')
<sos> Near The Portland Arms, The Cricketers is a coffee shop that also kid friendly and has a 3 out of 5 customer rating. <eos>

('name[Browns', 'Cambridge],', 'eatType[coffee', 'shop],', 'food[Chinese],', 'customer', 'rating[average],', 'area[riverside],', 'familyFriendly[yes],', 'near[Crowne', 'Plaza', 'Hotel]')
<sos> Near the Crowne Plaza Hotel in the riverside area is a family friendly coffee shop serving Chinese food. It has an average customer rating and is family friendly

In [182]:
decoder_type = "greedy"
for idx in [100, 120]:
    print(src_unique[idx])
    src = src_unique[idx]
    print("Geedy Search:")
    print(' '.join(get_cand(src_unique[idx], "greedy")))
    print(get_score(src, "greedy", ngram = ngram))
    print("Beam Search:")
    print(' '.join(get_cand(src_unique[idx], "beam")))
    print(get_score(src, "beam", ngram = ngram))
    print()

('name[Fitzbillies],', 'eatType[coffee', 'shop],', 'food[English],', 'priceRange[£20-25],', 'customer', 'rating[high],', 'area[city', 'centre],', 'familyFriendly[yes]')
Geedy Search:
<sos> Fitzbillies is a coffee shop shop in the city a price is <eos>
0.3383890558271442
Beam Search:
<sos> Fitzbillies has a high price range and is a child friendly coffee shop. It has an average customer rating and is located in the city centre. <eos>
0.2833593223209699

('name[The', 'Golden', 'Palace],', 'eatType[coffee', 'shop],', 'food[Chinese],', 'priceRange[more', 'than', '£30],', 'customer', 'rating[high],', 'area[city', 'centre]')
Geedy Search:
<sos> The Golden Palace is a high priced coffee shop located in the city Golden and serves high <eos>
0.5782419324934458
Beam Search:
There is a high priced coffee shop located in the city centre that serves Indian food. It has a high customer rating and is located near the Café in the riverside has a high price range and has a high customer rating. It has 